<a href="https://colab.research.google.com/github/mdpw/pytorch-deep-learning/blob/main/05_fcn_cifar_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR-10 Classification using Fully Connected Neural Network (FCN)

This notebook implements a Fully Connected Neural Network (FCN) using PyTorch to classify images from the CIFAR-10 dataset. This is intended to be a baseline for comparison with a Convolutional Neural Network (CNN) model.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


## Data Preparation

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


100%|██████████| 170M/170M [00:03<00:00, 46.7MB/s]


## Define the Fully Connected Neural Network

In [3]:
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 3*32*32)  # Flatten the image
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = FCN().to(device)


## Define Loss Function and Optimizer

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


## Train the Network

In [5]:
writer = SummaryWriter()
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            writer.add_scalar("training loss", running_loss / 100, epoch * len(trainloader) + i)
            running_loss = 0.0

    # Evaluate on test data after each epoch
    correct = 0
    total = 0
    with torch.no_grad():
        for test_data in testloader:
            test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
            test_outputs = net(test_images)
            _, predicted = torch.max(test_outputs.data, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1} Test Accuracy: {accuracy:.2f}%")
    writer.add_scalar("Test Accuracy", accuracy, epoch)
writer.close()
print("Finished Training")


[1, 100] loss: 1.880
[1, 200] loss: 1.685
[1, 300] loss: 1.635
[1, 400] loss: 1.559
[1, 500] loss: 1.546
Epoch 1 Test Accuracy: 45.66%
[2, 100] loss: 1.473
[2, 200] loss: 1.447
[2, 300] loss: 1.410
[2, 400] loss: 1.419
[2, 500] loss: 1.414
Epoch 2 Test Accuracy: 49.18%
[3, 100] loss: 1.311
[3, 200] loss: 1.322
[3, 300] loss: 1.310
[3, 400] loss: 1.295
[3, 500] loss: 1.322
Epoch 3 Test Accuracy: 51.36%
[4, 100] loss: 1.193
[4, 200] loss: 1.224
[4, 300] loss: 1.205
[4, 400] loss: 1.222
[4, 500] loss: 1.226
Epoch 4 Test Accuracy: 53.50%
[5, 100] loss: 1.085
[5, 200] loss: 1.123
[5, 300] loss: 1.129
[5, 400] loss: 1.111
[5, 500] loss: 1.136
Epoch 5 Test Accuracy: 54.19%
[6, 100] loss: 1.009
[6, 200] loss: 1.026
[6, 300] loss: 1.048
[6, 400] loss: 1.045
[6, 500] loss: 1.036
Epoch 6 Test Accuracy: 53.46%
[7, 100] loss: 0.890
[7, 200] loss: 0.908
[7, 300] loss: 0.964
[7, 400] loss: 0.973
[7, 500] loss: 0.981
Epoch 7 Test Accuracy: 53.65%
[8, 100] loss: 0.824
[8, 200] loss: 0.832
[8, 300] loss

## Evaluate on Test Data

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%")


Accuracy of the network on the 10000 test images: 54.37%
